In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')

In [4]:
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)


In [5]:
# Encode categorical vsriables
label_encoder_gender  = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [7]:
# One hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [27]:
# Combine one-hot encoded columns with orignal data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df],axis=1) 
data.head()

KeyError: "['Geography'] not found in axis"

In [ ]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [ ]:
## Split the data intraining and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
## Scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

## ANN Regressing Problem Statement

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) #Output layer for regrassing   
])

## Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

d:\ML course\ANN\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
# Set up early stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(
    X_train,y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100377.5781 - mae: 100377.5781 - val_loss: 98513.1016 - val_mae: 98513.1016
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99607.8438 - mae: 99607.8438 - val_loss: 96962.8750 - val_mae: 96962.8750
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 96909.3047 - mae: 96909.3047 - val_loss: 93008.9375 - val_mae: 93008.9375
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 91593.5859 - mae: 91593.5859 - val_loss: 86417.7578 - val_mae: 86417.7578
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 83865.6094 - mae: 83865.6094 - val_loss: 77910.7500 - val_mae: 77910.7500
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 74792.5859 - mae: 74792.5859 - val_loss: 69089.6484 - val_mae: 69089.6484
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 66025.2266 - mae: 66025.2266 - val_loss: 61399.6914 - val_mae: 61399.6914
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

In [ ]:
%load_ext tensorboard

In [28]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6008 (pid 36368), started 0:00:27 ago. (Use '!kill 36368' to kill it.)

In [29]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50309.3320 - mae: 50309.3320
Test MAE : 50309.33203125


In [30]:
model.save('regression_model.h5')